In [2]:
import pandas as pd
import numpy as np
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
import warnings
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM
from tensorflow.keras.utils import to_categorical
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, Callback
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
warnings.filterwarnings('ignore', category=DeprecationWarning)

#### Preprocessing

In [3]:
train, test = [], []
z_train, z_test = [], []
# chars =  ["bhA", "ba", "tA", "kaa", "tI"]
chars = ["hai", "ka", "rI", "sa", "pa"]

def read_ip(path):
    seq = []
    with open(path, 'r') as file:
        # Iterate over the lines in the file
        for line in file:
            data = [float(freq) for freq in line.split()]
            seq.append(data)
    return seq

for ch in chars:
    for f in os.listdir("Group24/CV_Data/" + str(ch)+"/Train"):
        train.append(read_ip("Group24/CV_Data/" + str(ch)+"/Train/" + str(f)))
        z_train.append(str(ch))
    for f in os.listdir("Group24/CV_Data/" + str(ch)+"/Test"):
        test.append(read_ip("Group24/CV_Data/" + str(ch) + "/Test/" + str(f)))
        z_test.append(str(ch))

In [4]:
train, val, z_train, z_val = train_test_split(train, z_train, test_size=0.2, random_state=42)

In [5]:
# label_map = {"bhA":0, "ba":1, "tA":2, "kaa":3, "tI":4}
label_map = {"hai":0, "ka":1, "rI":2, "sa":3, "pa":4}
z_train_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_train], 5)
z_val_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_val], 5)
z_test_onehot = tf.keras.utils.to_categorical([label_map[x] for x in z_test], 5)

In [6]:
train_tensor = tf.ragged.constant(train)
test_tensor = tf.ragged.constant(test)
val_tensor = tf.ragged.constant(val)

2023-05-09 17:11:49.817051: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-05-09 17:11:49.817078: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-05-09 17:11:49.817095: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (piyush-HP-Spectre-x360-Convertible-13-aw2xxx): /proc/driver/nvidia/version does not exist
2023-05-09 17:11:49.817355: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


#### Model

In [7]:
model = Sequential()
model.add(SimpleRNN(256, batch_input_shape=(None, None, 39), return_sequences=True))
model.add(SimpleRNN(128, activation="relu", return_sequences=True))
model.add(SimpleRNN(64, activation="relu", return_sequences=True))
model.add(SimpleRNN(32, activation="relu", return_sequences=False))
model.add(Dense(64, activation="relu"))
model.add(Dense(5, activation="softmax"))
loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 
# Train the model
my_callbacks = [ EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=20),    TensorBoard(log_dir=f'./logdir/Q1/test')]
model_fit = model.fit(train_tensor, z_train_onehot, validation_data=(val_tensor, z_val_onehot), batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/test.tf')

loss, accuracy = model.evaluate(test_tensor,z_test_onehot)

print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

#### LSTM Model

In [9]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, batch_input_shape=(None,None,39)),
    tf.keras.layers.Dense(5, activation='softmax')
])

loss_fn = tf.keras.losses.CategoricalCrossentropy()

adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
model.compile(loss="categorical_crossentropy", optimizer=adam_optimizer, metrics=['accuracy']) 
# Train the model
my_callbacks = [ EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=20),    TensorBoard(log_dir=f'./logdir/Q1/test')]
model_fit = model.fit(train_tensor, z_train_onehot, validation_data=(val_tensor, z_val_onehot), batch_size=len(train), epochs=10000, verbose=1, validation_split=0.0,callbacks=my_callbacks, shuffle=False, validation_batch_size=None)

hist_metric = 'accuracy'
print(f'epochs: {len(model_fit.history[hist_metric])}, acc: {model_fit.history[hist_metric][-1]}\n')
model.save(f'models/Q1/test.tf')

loss, accuracy = model.evaluate(test_tensor,z_test_onehot)

print("Test loss = ", loss)
print("Test accuracy = ", accuracy)

Epoch 1/10000
1/1 [==============================] - 2s 2s/step - loss: 1.7985 - accuracy: 0.2504 - val_loss: 1.7435 - val_accuracy: 0.2415
Epoch 2/10000
1/1 [==============================] - 0s 208ms/step - loss: 1.7281 - accuracy: 0.2795 - val_loss: 1.6808 - val_accuracy: 0.2614
Epoch 3/10000
1/1 [==============================] - 0s 213ms/step - loss: 1.6622 - accuracy: 0.3009 - val_loss: 1.6225 - val_accuracy: 0.2955
Epoch 4/10000
1/1 [==============================] - 0s 208ms/step - loss: 1.6017 - accuracy: 0.3257 - val_loss: 1.5679 - val_accuracy: 0.3381
Epoch 5/10000
1/1 [==============================] - 0s 209ms/step - loss: 1.5462 - accuracy: 0.3542 - val_loss: 1.5162 - val_accuracy: 0.3892
Epoch 6/10000
1/1 [==============================] - 0s 226ms/step - loss: 1.4948 - accuracy: 0.3876 - val_loss: 1.4676 - val_accuracy: 0.4062
Epoch 7/10000
1/1 [==============================] - 0s 208ms/step - loss: 1.4471 - accuracy: 0.4203 - val_loss: 1.4227 - val_accuracy: 0.4403
Ep

INFO:tensorflow:Assets written to: models/Q1/test.tf/assets


INFO:tensorflow:Assets written to: models/Q1/test.tf/assets


14/14 [==============================] - 0s 7ms/step - loss: 0.3845 - accuracy: 0.8821
Test loss =  0.38450300693511963
Test accuracy =  0.8820861577987671
